In [7]:
%load_ext autoreload
%autoreload 2


In [12]:
# Connect to the OntoTransServer ..
from otelib.ontotransserver import OntoTransServer
server = OntoTransServer('http://localhost:80/api/v1')
#server = OntoTransServer('<OTEAPI_ENDPOINT>') 

In [10]:

import requests

In [13]:
response=requests.post("http://localhost:8080/api/v1/datasource/", data={"downloadUrl": "https://filesamples.com/samples/code/json/sample2.json", "mediaType": "text/json", "accessUrl": None, "accessService": None, "license": None, "accessRights": None, "description": None, "published": None, "configuration": None})
print(response.reason)
response

ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /api/v1/datasource/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f3424f9c5b0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [14]:
data = server.create_dataresource(downloadUrl="https://filesamples.com/samples/code/json/sample2.json", 
                                  mediaType="text/json")

ApiError: APIError: status=Cannot create dataresouce: 404

In [3]:
database = {"h2": "https://raw.githubusercontent.com/susilehtola/erkale/master/tests/xyz/H2.xyz", 
            "ethene": "https://raw.githubusercontent.com/nutjunkie/IQmol/master/share/fragments/Molecules/Alkenes/Ethene.xyz",
            "ethane": "https://raw.githubusercontent.com/nutjunkie/IQmol/master/share/fragments/Molecules/Alkanes/Ethane.xyz",
            "methane": "https://raw.githubusercontent.com/nutjunkie/IQmol/master/share/fragments/Molecules/Alkanes/Methane.xyz"}

In [4]:
# NB: Dette er juks, jeg bruker feil identifier. Cif fil (men ase som vi bruker til å lese xyz kan også lese cif)

downloadDinitroanisole = server.create_dataresource(downloadUrl="http://www.crystallography.net/cod/7220950.cif?CODSESSION=uarsa75qn9gnepakhm9g7jikl1",
                                                    mediaType="chemical/x-xyz")
# Instantiate Transformation strategy for calculating molecular energies 
calcMoleculeTransformation = server.create_transformation(transformation_type="ase/calc-energy")

ApiError: APIError: status=Cannot create dataresouce: 422

In [6]:
# Build a pipeline
pipeline = downloadDinitroanisole >> calcMoleculeTransformation
#Excecute a pipeline and print result
print(pipeline.get())

session-ebb90b30-b93d-4dda-9b09-ca3ae428a587
b'{"collection_id":"a0dc8b97-c240-47af-8c7c-ce6d60e80e3e"}'


# IDEA #

Calculate reaction A + B -> C 

reaction = {A: -1, B: -1, C, +1} 

1) obtain chemical structures of A, B, C # parse-strategy includes download
2) calculate "electronic" energies of A, B, C # transformation strategy
3) calculate reaction energy # transformation strategy

Using Dlite collections 


In [7]:
# Instantiate Download Molecule strategies
key="h2"
downloadHydrogen = server.create_dataresource(
    downloadUrl=database[key],
    mediaType="chemical/x-xyz",
    configuration={"moleculeName":key})

key = "ethane"
downloadEthane = server.create_dataresource(
    downloadUrl=database[key],
    mediaType="chemical/x-xyz",
    configuration={"moleculeName":key})

key = "ethene"
downloadEthene = server.create_dataresource(
    downloadUrl=database[key],
    mediaType="chemical/x-xyz",
    configuration={"moleculeName":key})

# Instantiate Transformation strategy for calculating molecular energies 
calcMoleculeTransformation1 = server.create_transformation(transformation_type="ase/calc-energy")
calcMoleculeTransformation2 = server.create_transformation(transformation_type="ase/calc-energy")
calcMoleculeTransformation3 = server.create_transformation(transformation_type="ase/calc-energy")

# Instantiate Transformation strategy for calculating reaction energy
reaction = {'Ethene':1, 'H2': 1,'Ethane':-1}
calcReactionTransformation = server.create_transformation(transformation_type="reaction/calc-reaction-energy",
                                                          configuration={"stoichiometricReactants":reaction})


# Instantiate getResult strategy 
#getResult = server.create_transformation(transformation_type= "dlite/compevo-result")

In [8]:
pipeline = downloadHydrogen >> calcMoleculeTransformation1 >>  downloadEthene >> calcMoleculeTransformation2 >> downloadEthane >> calcMoleculeTransformation3 >> calcReactionTransformation
pipeline.get()

session-191118a8-6a64-4279-adf5-05ef0c6e3bdd


b'{"reaction_energy":0.5447873775858421}'

# What I wish for 

SmilesStr -> TransformToXYZ -> PARSEXYZ_uten_download(til dliteMolecule)
eller
DownloadXYZ+PARSE

# Also, reuse the same calcMoleculeTransformation
downloadHydrogen >> calcMoleculeTransformation  \ 
downloadEthane >> calcMoleculeTransformation      --- >> CalcReactionTransformation 
downloadEthene >> calcMoleculeTransformation _/


In [15]:
from ase import Atoms, Atom
from ase.io import read
from ase.visualize import view

In [36]:
a = read("7220950.cif")
view(a, viewer='x3d')

In [ ]:
import rdkit

In [ ]:

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors

In [9]:
reaction = {'ethene':1, 'h2':1, 'ethane':-1}

download_strategies = [server.create_dataresource(
                          downloadUrl=database[key],
                          mediaType="chemical/x-xyz",
                          configuration={"moleculeName":key}) for key in reaction.keys()]

transformation_strategies = [server.create_transformation(transformation_type="ase/calc-energy") for key in reaction.keys()]


calcReactionTransformation = server.create_transformation(transformation_type="reaction/calc-reaction-energy",
                                                          configuration={"stoichiometricReactants":reaction})

In [16]:
download_strategies[0] >> transformation_strategies[0]
len(reaction)

3

In [17]:
pipelines = [download_strategies[i] >> transformation_strategies[i] for i in range(len(reaction))]
pipeline = [pipelines[i] 

In [57]:
pipeline = download_strategies[0] >> transformation_strategies[0] >> download_strategies[1] >> transformation_strategies[1] >> download_strategies[2] >> transformation_strategies[2] >> calcReactionTransformation

pipeline.get()
    

session-7bb6eea4-b10b-4357-b28d-7a0e28fed331


b'Internal Server Error'